In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 50
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
from Tars.distributions import Categorical
from Tars.models import ML

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
x_dim = 784
y_dim = 10

# discriminative model p(y|x)
class Discriminative(Categorical):
    def __init__(self):
        super(Discriminative, self).__init__(cond_var=["x"], var=["y"])
        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, y_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        h = F.softmax(self.fc3(h), dim=1)
        
        return {"probs": h}

In [5]:
p = Discriminative()
p.to(device)

Discriminative(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)

In [6]:
model = ML(p, optim.Adam, {"lr":1e-3})

In [7]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data_x, data_y) in enumerate(tqdm(train_loader)):
        data_x = data_x.view(-1, 784).to(device)
        data_y = torch.eye(10)[data_y].to(device)
        log_like, loss = model.train({"x": data_x, "y": data_y})
        train_loss += loss
        
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    
    return train_loss

In [8]:
def test(epoch):
    test_loss = 0
    for i, (data_x, data_y) in enumerate(test_loader):
        data_x = data_x.view(-1, 784).to(device)
        data_y = torch.eye(10)[data_y].to(device)
        log_like, loss = model.test({"x": data_x, "y": data_y})
        test_loss += loss
        
    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    
    return test_loss

In [9]:
writer = SummaryWriter()

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)     
    
writer.close()

100%|██████████| 469/469 [00:05<00:00, 85.56it/s]

Epoch: 1 Train loss: 0.2704



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 0.1224


100%|██████████| 469/469 [00:06<00:00, 77.29it/s]

...


  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 0.1063


100%|██████████| 469/469 [00:05<00:00, 78.40it/s]

Epoch: 50 Train loss: 0.0003


Test loss: 0.1063
